Pythonを用いて惑星の運動に関するシミュレーションを行う。  
まず、極座標系における運動方程式は、以下で与えられるのであった。

$$
\displaystyle
\left\{
    \begin{align*}
        m
        \left(
            \frac{d^{2}r}{dt^{2}}
            -
            r 
            \left(
                \frac{d\theta}{dt}
            \right)
            ^{2}
        \right)
        =
        F_{r} \\
        m
        \frac{1}{r} 
        \frac{d}{dt}
        \left(
            r^{2}
            \frac{d\theta}{dt}
        \right)
        = 
        F_{\theta}
    \end{align*}
\right.
$$

ここで、惑星の周回においては、
$$
\displaystyle
\left\{
    \begin{align*}
        F_{r} &= -\frac{GmM}{r^{2}} \\
        F_{\theta} &= 0
    \end{align*}
\right.
$$
が成り立つから、適当な定数A, Bを用いて、

$$
\displaystyle
\left\{
    \begin{align*}
        \frac{d^{2}r}{dt^{2}}
        -
        r 
        \left(
            \frac{d\theta}{dt}
        \right)
        ^{2}
        &=
        \frac{A}{r^{2}} \\
        \frac{d}{dt}
        \left(
            r^{2}
            \frac{d\theta}{dt}
        \right)
        &= 
        0 
        \Rightarrow
            r^{2}
            \frac{d\theta}{dt} = B
    \end{align*}
\right.
$$

と書ける。また、定数C, D, Eを用いて、初期条件を以下で定める。

$$
\displaystyle
\left\{
    \begin{align*}
        r(0) &= C \\
        \theta(0) &= D \\
        \frac{dr}{dt}(0) &=E \\
        \frac{d\theta}{dt}(0) &= \frac{B}{C^{2}}
    \end{align*}
\right.
$$

この仮定をもとに、Scipyのsolve_ivp()を用いて微分方程式を解く。

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from scipy.integrate import solve_ivp
from IPython.display import HTML
from matplotlib.animation import ArtistAnimation

# 微分方程式
def f(t, y, a, b):
    r, th, r_dot, th_dot = y
    th_dot = b/r**2
    r_dotdot = a/r**2 + r * th_dot**2
    th_dotdot = - b*2*r_dot/r**3
    return np.array([r_dot, th_dot, r_dotdot, th_dotdot])

# 初期条件
a = -2.0 
b = 1.9
c = 1.1
d = 0.0
e = 0.0
ini = np.array([c, d, e, b/c**2])

# 時間の範囲に関する制約
t_min = 0
t_max = 50
dt = 0.1
t = np.arange(t_min, t_max, dt)
t_span = (t_min, t_max)

# Solve
solved_data = solve_ivp(f, t_span, ini, t_eval=t, args=(a, b))

# アニメーションの描画
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.set_xlim([-6.0, 6.0]) 
ax.set_ylim([-6.0, 6.0]) 
ax.grid(True)
artists = []
i = 0
while not(solved_data.y[1][i] > 2 * np.pi) and (i < int(t_max / dt) - 1):
    i += 1

orbit = ax.plot(solved_data.y[0][0:i+1]*np.cos(solved_data.y[1][0:i+1]),
    solved_data.y[0][0:i+1]*np.sin(solved_data.y[1][0:i+1]), color="gray")
star = ax.plot(0, 0, "o", color="red")

for j in range(i):
    planet = ax.plot(solved_data.y[0][j]*np.cos(solved_data.y[1][j]),
        solved_data.y[0][j]*np.sin(solved_data.y[1][j]), "o", color="blue")
    artists.append(orbit+star+planet)

ax.set(xlabel="x", ylabel="y")
anim = ArtistAnimation(fig, artists, interval=30)
plt.close()
HTML(anim.to_html5_video())